In [ ]:
import torch
import numpy as np
from itertools import compress
from models.yolo.models import Darknet
from models.yolo.utils.utils import non_max_suppression, load_classes
from torch.nn.utils.rnn import pad_sequence

MODEL_DEF = "models/yolo/config/yolov3.cfg"
WEIGHTS_PATH = "models/yolo/weights/yolov3.weights"
CLASS_PATH = "models/yolo/data/coco.names"
CONF_THRES = 0.8
NMS_THRES = 0.4
IMG_SIZE = 416
CATEGORY_NAME = 'person'

class YoloModel(torch.nn.Module):

    def __init__(self, device):
        super(YoloModel, self).__init__()
        self.device = device
        self.model = Darknet(MODEL_DEF, img_size=IMG_SIZE).to(self.device)
        self.model.load_darknet_weights(WEIGHTS_PATH)
        self.model.eval()
        self.classes = load_classes(CLASS_PATH)


    def forward(self, batch):
        detections = self.model(batch)
        detections = non_max_suppression(detections, CONF_THRES, NMS_THRES)
        self.ids, self._probs, self.probs, self.pred_classes, self.is_ok = [], [], [], [], []

        for det in detections:
            if det is None:
                self._save_results(False, [], [], [])
                continue
            ids, probs, pred_classes = self._extract_results(det)
            if not CATEGORY_NAME in pred_classes:
                self._save_results(False, [], [], [])
                continue
            self._save_results(True, ids, pred_classes, probs)

        if not self._probs:
            return torch.tensor([-1])

        self.id_pos = [x.index(CATEGORY_NAME) for x in list(compress(self.pred_classes, self.is_ok))]
        self._probs = pad_sequence(self._probs)
        self._probs = torch.transpose(self._probs, 0, 1)
        return self._probs

    def get_probs(self):
        return self.probs

    def get_ids(self):
        return self.ids

    def get_classes(self):
        return self.pred_classes

    def get_category_id_pos(self):
        return self.id_pos

    def get_ok_list(self):
        return self.is_ok

    def is_backward_ready(self):
        return False

    def _extract_results(self, det):
        det = det[:, 5:]
        det = det[det[:, 0].argsort()]
        det = torch.flip(det, [0])
        ids = det[:, 1:].squeeze(1)
        probs = det[:, :1].squeeze(1)
        pred_classes = [self.classes[x] for x in ids.long().detach().cpu().numpy()]
        return ids, probs, pred_classes

    def _save_results(self, is_ok, ids, pred_classes, probs):
        self.is_ok.append(is_ok)
        self.ids.append(ids)
        self.pred_classes.append(pred_classes)
        self.probs.append(probs)
        if is_ok:
            self._probs.append(probs)

In [ ]:
import numpy as np
from torchvision import transforms
import cv2
from PIL import Image
from torch.utils.data import Dataset
from data.cocoapi_master.PythonAPI.pycocotools.coco import COCO
from models.yolo.utils.datasets import pad_to_square, resize
from models.yolo.utils.utils import load_classes

ANNOTATION_FILEPATH = '/visinf/projects_students/shared_vqa/mscoco/coco-annotations/instances_train2014.json'
DATASET_PATH = '/visinf/projects_students/shared_vqa/mscoco/train2014/'
CLASS_PATH = "models/yolo/data/coco.names"
IMG_SIZE = 416
CATEGORY_NAME = 'person'

class CocoYoloDataset(Dataset):
    def __init__(self, device):
        self.annotation_filepath = ANNOTATION_FILEPATH
        self.dataset_path = DATASET_PATH
        self.device = device

        # initialize COCO api for instance annotations
        self.coco=COCO(self.annotation_filepath)
        self.imgIds = self.coco.getImgIds(catIds=[self._get_category_id(CATEGORY_NAME)])

    def __len__(self):
        return len(self.imgIds)

    def __getitem__(self, index):
        if index == 10035: # The corresponding image is corrupted
            index = 10034
        img_id = self.imgIds[index]
        img_infos = self.coco.loadImgs([img_id])[0]
        filepath = self.dataset_path+img_infos['file_name']
        img = Image.open(filepath)
        if len(np.shape(img)) == 2:
            img = img.convert('RGB')
        gt = self._get_ground_truth(img=img, img_id=img_id, category_name=CATEGORY_NAME)
        img = transforms.ToTensor()(img)
        img, _ = pad_to_square(img, 0)
        img = resize(img, IMG_SIZE)
        img = img.to(self.device)

        return {"img": img, "gt": gt, "filepath": filepath}

    def _get_ground_truth(self, img, img_id, category_name=""):
        ground_truth = np.zeros((np.shape(img)[0], np.shape(img)[1]))
        contours = self._get_contours(img_id, category_name)
        for contour in contours:
            contour = contour.astype('int32')
            cv2.fillPoly(ground_truth, [contour], 255)
        ground_truth = cv2.resize(ground_truth, (IMG_SIZE, IMG_SIZE))
        ground_truth = np.array(ground_truth, dtype=np.uint8)
        ground_truth[ground_truth > 0] = 1
        return ground_truth

    def _get_contours(self, img_id, category_name):
        if category_name == "":
            annIds = self.coco.getAnnIds(imgIds=[img_id])
        else:
            annIds = self.coco.getAnnIds(imgIds=[img_id], catIds=[self._get_category_id(category_name)])
        anns = self.coco.loadAnns(annIds)
        contours = []
        for ann in anns:
            if 'segmentation' in ann and type(ann['segmentation']) == list:
                for seg in ann['segmentation']:
                    contour = np.array(seg).reshape((int(len(seg)/2), 2))
                    contours.append(contour)
        return np.asarray(contours)

    def _get_category_id(self, category_name):
        cats = self.coco.loadCats(self.coco.getCatIds())
        for cat in cats:
            if cat['name'] == category_name:
                return cat['id']

In [ ]:
import evaluate_grad_cam

DEVICE = "cuda"
dataset = CocoYoloDataset(device=DEVICE)
model = YoloModel(device=DEVICE)

evaluate_grad_cam.evaluate_dataset(model, dataset)
